In [1]:
import pickle
import pandas as pd
import json

In [2]:
players_dict = {}
with open('data/players.pkl', 'rb') as f:
    players_dict = pickle.load(f)

players_df = pd.DataFrame(players_dict).transpose()
print(players_df)

            id       name  patronymic    surname
1            1    Алексей        None   Абабилов
10          10      Игорь                 Абалов
11          11    Наталья     Юрьевна  Абалымова
12          12      Артур  Евгеньевич    Абальян
13          13       Эрик  Евгеньевич    Абальян
...        ...        ...         ...        ...
224700  224700      Артём  Евгеньевич      Садов
224701  224701     Даниил    Олегович   Трефилов
224702  224702   Владимир  Араратович   Басенцян
224703  224703     Руслан   Ринатович   Дауранов
224704  224704  Александр  Викторович    Гапонов

[204063 rows x 4 columns]


In [3]:
tournaments_dict = {}
with open('data/tournaments.pkl', 'rb') as f:
    tournaments_dict = pickle.load(f)

tournaments_df = pd.DataFrame(tournaments_dict).transpose()
print(tournaments_df)

        id                                               name  \
1        1                           Чемпионат Южного Кавказа   
2        2                                        Летние зори   
3        3                                   Турнир в Ижевске   
4        4                 Чемпионат Украины. Переходной этап   
5        5                                Бостонское чаепитие   
...    ...                                                ...   
6481  6481  Онлайн: 15:00 (а)Синхрон-lite. Лига старта. Эп...   
6482  6482                          Онлайн: 19:00 Зелёный шум   
6483  6483  Онлайн: 19:00 (а)Синхрон-lite. Лига старта. Эп...   
6484  6484      Онлайн: 22:00 Не числом, а умением - 2 (NEW!)   
6485  6485                 Онлайн: 19:00 Не числом, а умением   

                      dateStart                    dateEnd  \
1     2003-07-25T00:00:00+04:00  2003-07-27T00:00:00+04:00   
2     2003-08-09T00:00:00+04:00  2003-08-09T00:00:00+04:00   
3     2003-11-22T00:00:00+03:00  

In [4]:
results_dict = {}
with open('data/results.pkl', 'rb') as f:
    results_dict = pickle.load(f)

In [5]:
valid_train_results = {}
for i, tournament in tournaments_df.iterrows():
    if not tournament.empty and tournament['dateStart'] != None:
        if int(tournament['dateStart'][:4]) == 2019:
            valid_participants = []
            for participant in results_dict[i]:
                if participant:
                    if 'mask' in participant and participant['mask']:
                        if 'teamMembers' in participant and participant['teamMembers']:
                            valid_participants.append(participant)
            if valid_participants:
                valid_train_results[i] = valid_participants

In [6]:
valid_test_results = {}
for i, tournament in tournaments_df.iterrows():
    if not tournament.empty and tournament['dateStart'] != None:
        if int(tournament['dateStart'][:4]) == 2020:
            valid_participants = []
            for participant in results_dict[i]:
                if participant:
                    if 'mask' in participant and participant['mask']:
                        if 'teamMembers' in participant and participant['teamMembers']:
                            valid_participants.append(participant)
            if valid_participants:
                valid_test_results[i] = valid_participants

In [7]:
with open('data/valid_train_results.json', 'w') as fin:
    json.dump(valid_train_results, fin)

In [8]:
with open('data/valid_test_results.json', 'w') as fin:
    json.dump(valid_test_results, fin)

In [9]:
x_train = pd.DataFrame(columns = ['player_id', 'tournament_id','question_id', 'team_id'])
y_train = pd.DataFrame(columns = ['hit'])

x_test = pd.DataFrame(columns = ['player_id', 'tournament_id', 'question_id', 'team_id'])
y_test = pd.DataFrame(columns = ['hit'])

In [10]:
file_x_train = open('data/x_train.csv', 'w')
file_x_train.write('player_id,tournament_id,question_id\n')

file_y_train = open('data/y_train.csv', 'w')
file_y_train.write('hit\n')

for tournament_id in valid_train_results:
    tournament = valid_train_results[tournament_id]
    for team in tournament:
        team_id = team['team']['id']
        for teammate in team['teamMembers']:
            player_id = teammate['player']['id']
            for question_id in range(1, len(team['mask']) + 1):
                file_x_train.write(str(player_id) + ',' + str(tournament_id) + ',' + str(question_id) + ',' + str(team_id) + '\n')
                file_y_train.write(team['mask'][question_id - 1] + '\n')

file_x_train.close()
file_y_train.close()

In [11]:
file_x_test = open('data/x_test.csv', 'w')
file_x_test.write('player_id,tournament_id,question_id\n')

file_y_test = open('data/y_test.csv', 'w')
file_y_test.write('hit\n')

for tournament_id in valid_test_results:
    tournament = valid_test_results[tournament_id]
    for team in tournament:
        team_id = team['team']['id']
        for teammate in team['teamMembers']:
            player_id = teammate['player']['id']
            for question_id in range(1, len(team['mask']) + 1):
                file_x_test.write(str(player_id) + ',' + str(tournament_id) + ',' + str(question_id) + ',' + str(team_id) + '\n')
                file_y_test.write(team['mask'][question_id - 1] + '\n')

file_x_test.close()
file_y_test.close()